# earthaccess and NASA EDL

earthaccess allows us to access data from datasets behind NASA EDL. This library comes with handy methods to generate an access token, create an authenticated Python requests session or fsspec file accessors. 

The following are simple examples of what can we do with them.

In [1]:
import earthaccess

auth = earthaccess.login()

### Data in AWS
If the data we want to access is on AWS, we can use earthaccess to generate temporary S3 credentials for any of the DAACs. This line is commented out for security reasons. 

In [2]:
#s3_credentials = auth.get_s3_credentials("NSIDC")

These S3 temporary credentials are valid for 1 hour and can be used by third party libraries that support S3 buckets.

### HTTPS access

We can also access data over HTTP using presigned Python `requests` sessions. The advantage of these sessions is that they work on every DAAC or data in S3 when accessed through HTTPS.

In [3]:
nsidc_url = "https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL06.005/2019.02.21/ATL06_20190221121851_08410203_005_01.h5"
lpcloud_url = "https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc"

# this is a Python requests session 
session = earthaccess.get_requests_https_session()

In [4]:
headers = {"Range": "bytes=0-100"}
r = session.get(lpcloud_url, headers=headers)
r.text

'�HDF\r\n\x1a\n\x00\x00\x00\x00\x00\x08\x08\x00\x04\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00���������HUn\x00\x00\x00\x00��������\x00\x00\x00\x00\x00\x00\x00\x00`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00OHDR\x02'

### Accessing remote files as if they were local with fsspec

In [5]:
fs = earthaccess.get_fsspec_https_session()

In [6]:
with fs.open(lpcloud_url) as f:
    data = f.read(100)
data

b'\x89HDF\r\n\x1a\n\x00\x00\x00\x00\x00\x08\x08\x00\x04\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xd7HUn\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00OHDR'

In [7]:
%%time 

import xarray as xr

# earthaccess can open a list of files
files = earthaccess.open([lpcloud_url])

ds = xr.open_dataset(files[0], group="sensor_band_parameters")
ds

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 1.4 s, sys: 179 ms, total: 1.58 s
Wall time: 10.5 s


<xarray.Dataset> Size: 3kB
Dimensions:           (bands: 285)
Dimensions without coordinates: bands
Data variables:
    wavelengths       (bands) float32 1kB ...
    fwhm              (bands) float32 1kB ...
    good_wavelengths  (bands) float32 1kB ...